### Reference:
1. Deeplizard

### Notes:
1. How to run tensorboard: 

### Next topic: 
1. Compare runs. This is the real advantage of tensorboard, since python can already plot these data.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

torch.set_printoptions(linewidth=120)

In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [6]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        def forward(self, t):
            t = F.relu(self.conv1(t))
            t = F.max_pool2d(t, kernel_size=2, stride=2)
                       
            t = F.relu(self.conv2(t))
            t = F.max_pool2d(t, kernel_size=2, stride=2)
                       
            t = t.flatten(start_dim=1)
            t = F.relu(self.fc1(t))
                       
            t = F.relu(self.fc2(t))
            
            t = self.out(t)
            return t

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

NameError: name 'train_set' is not defined

Tensorboard

In [8]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

Training Loop Review

In [ ]:
tb = SummaryWriter()

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader:
        images, labels = batch
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        
        optimizer.zero_grad()
        loss.backward() # Compute gradients
        optimizer.step() # Update weights
        
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)
    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.grad', network.conv1.weight.grad, epoch)
    
    print('epoch', epoch, 'total_correct', total_correct, 'loss', total_loss)
    
tb.close()